# OpenCV 이미지 변환 실습 노트북

본 노트북에서는 Google Colab 환경에서 **OpenCV** 라이브러리를 활용하여 다음 내용을 실습합니다.

1. **이미지 크기 조절(Resizing)** + 다양한 **보간법(Interpolation)**
2. **이미지 위치 변경(Translation)**
3. **이미지 회전(Rotation)**
4. 추가 **과제(연습 문제)** 제시 및 예시 결과

각 함수별 사용법과 주요 파라미터에 대한 설명이 포함되어 있으니, 따라 해보면서 OpenCV의 기초 이미지 변환 기능을 익혀 보세요.

## 1. 환경 설정 및 라이브러리 불러오기
Colab에서 OpenCV가 미설치된 경우를 대비해 설치 명령어를 먼저 실행한 뒤, 필요한 라이브러리를 임포트합니다.

In [ ]:
!pip install opencv-python-headless

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import files

print("OpenCV version:", cv2.__version__)

## 2. 이미지 업로드 & 원본 이미지 확인
Colab에서 이미지를 직접 업로드하고, **cv2_imshow**를 통해 이미지를 확인해 봅니다.

```python
files.upload()  # 사용자가 이미지 파일을 업로드할 수 있는 창이 뜹니다.
```
이후 업로드된 파일 이름을 확인한 뒤, `cv2.imread`로 이미지를 읽어옵니다.

In [ ]:
# 이미지 업로드
uploaded = files.upload()

# 업로드된 첫 번째 이미지 파일 이름 가져오기
filename = list(uploaded.keys())[0]

# 이미지를 BGR 컬러로 읽기
image = cv2.imread(filename)

# 원본 이미지 확인
print("[원본 이미지]")
cv2_imshow(image)
print("이미지 크기:", image.shape)

# 3. 이미지 크기 조절 (Resizing)
이미지의 크기를 변경(확대/축소)할 때에는 **`cv2.resize()`** 함수를 사용합니다.

```python
resized = cv2.resize(
    src,                # 원본 이미지
    dsize,             # (width, height) - 직접 지정하거나 None으로 둘 수 있음
    fx=1.0,            # 가로 방향 배율
    fy=1.0,            # 세로 방향 배율
    interpolation=cv2.INTER_LINEAR  # 보간법(기본값: INTER_LINEAR)
)
```

### 보간법(Interpolation)
이미지를 확대/축소할 때, 실제 존재하지 않는 픽셀 사이 값을 채우거나 제거해야 합니다. 이때 **어떻게** 계산해서 값을 구할지가 보간법으로 결정됩니다.

1. **`cv2.INTER_NEAREST`**
   - 최근접 이웃 보간법.
   - 가장 단순하고 빠르지만, 이미지 품질이 거칠 수 있으며 계단 현상이 두드러질 수 있습니다.
2. **`cv2.INTER_LINEAR`** (기본값)
   - 양선형 보간법.
   - 확대, 축소 모두 적절한 품질과 속도를 제공합니다.
3. **`cv2.INTER_CUBIC`**
   - 4×4 이웃 픽셀을 고려하는 삼차 보간법.
   - 확대 시 매우 높은 품질을 제공하지만, 계산 비용이 더 큽니다.
4. **`cv2.INTER_AREA`**
   - 영역 보간법.
   - 주로 축소 시에 좋은 결과를 냅니다(평균화 과정).
5. **`cv2.INTER_LANCZOS4`**
   - Lanczos 보간법.
   - 8×8 이웃 픽셀을 고려하여 고품질 확대/축소를 구현합니다.

아래 코드에서는 원본 이미지를 **2배 확대**하여 여러 보간법별 결과를 비교해 봅니다.

In [ ]:
print("\n[보간법을 적용한 이미지 크기 조절 예시 - 2배 확대]")
# 보간법 테스트용 딕셔너리
methods = {
    "INTER_NEAREST (최근접 이웃)": cv2.INTER_NEAREST,
    "INTER_LINEAR (양선형)": cv2.INTER_LINEAR,
    "INTER_CUBIC (삼차)": cv2.INTER_CUBIC,
    "INTER_AREA (영역)": cv2.INTER_AREA,
    "INTER_LANCZOS4": cv2.INTER_LANCZOS4
}

for name, method in methods.items():
    resized = cv2.resize(
        image,
        None,               # dsize를 None으로 두면 fx, fy 배율로 크기 결정
        fx=2.0,            # 가로 2배
        fy=2.0,            # 세로 2배
        interpolation=method
    )
    print(name)
    cv2_imshow(resized)

### 크기 축소 예시
이미지를 축소할 때는 보통 **`INTER_AREA`** 보간법이 좋은 결과를 보인다고 알려져 있습니다. 아래에서는 0.8배 축소를 시도합니다.

In [ ]:
print("[이미지 축소: 0.8배, INTER_AREA 보간법]")
shrink = cv2.resize(
    image,
    None,
    fx=0.8,
    fy=0.8,
    interpolation=cv2.INTER_AREA
)
cv2_imshow(shrink)
print("축소 후 이미지 크기:", shrink.shape)

# 4. 이미지 위치 변경 (Translation)
이미지를 평면 상에서 **옮기는(이동시키는)** 방법입니다.
OpenCV에서는 **`cv2.warpAffine()`** 함수를 사용하며, 이를 위해 이동 변환 행렬(**Translation Matrix**)을 만들어야 합니다.

```python
M = np.float32([
    [1, 0, tx],  # x 방향 이동
    [0, 1, ty]   # y 방향 이동
])

dst = cv2.warpAffine(
    src=image,
    M=M,               # 변환 행렬
    dsize=(width, height)  # 결과 이미지 크기
)
```

- **`tx`**: x축으로 이동하는 픽셀 수 (양수 → 오른쪽 이동, 음수 → 왼쪽 이동)
- **`ty`**: y축으로 이동하는 픽셀 수 (양수 → 아래쪽 이동, 음수 → 위쪽 이동)
- **`dsize`**: 결과 이미지의 크기. 일반적으로 원본과 같은 크기를 지정합니다.

In [ ]:
# 이미지의 높이, 너비
height, width = image.shape[:2]

# 이동 변환 행렬 (x: +50 → 오른쪽 50px, y: +10 → 아래쪽 10px)
M = np.float32([[1, 0, 50], [0, 1, 10]])

# 이동 적용
translated = cv2.warpAffine(
    image,  # 원본 이미지
    M,      # 변환 행렬
    (width, height)  # 출력 이미지 크기 (가로, 세로)
)

print("[이미지 위치 변경: (x:+50, y:+10)]")
cv2_imshow(translated)

## 5. 이미지 회전 (Rotation)
이미지를 특정 각도로 회전시키는 방법입니다. 마찬가지로 **`cv2.warpAffine()`** 함수를 사용하지만, **`cv2.getRotationMatrix2D()`**를 이용해 회전 행렬을 먼저 구해야 합니다.

```python
M = cv2.getRotationMatrix2D(
    center,   # 회전 중심 (tuple)
    angle,    # 회전 각도 (도 단위, +값 = 반시계방향)
    scale     # 크기 비율 (1.0이면 원본 크기 유지)
)

rotated = cv2.warpAffine(image, M, (width, height))
```

- **`center`**: (cx, cy) 형태로 회전의 중심점 좌표
- **`angle`**: 회전 각도(+일 때 반시계 방향)
- **`scale`**: 회전하면서 이미지 크기를 확대/축소하는 비율(1.0 → 원본)

아래 예제에서는 이미지의 중심을 기준으로 90도 회전하고, 크기는 절반(0.5)으로 축소합니다.

In [ ]:
# 이미지의 높이, 너비
height, width = image.shape[:2]

# 회전 중심 (이미지의 중앙)
center = (width / 2, height / 2)

# 90도 회전 + 0.5 배 스케일 (반시계방향)
M = cv2.getRotationMatrix2D(
    center=center,
    angle=90,
    scale=0.5
)

# 회전 적용
rotated = cv2.warpAffine(
    image,  # 원본 이미지
    M,      # 회전 행렬
    (width, height)  # 출력 이미지 크기
)

print("[이미지 회전: 90도, 스케일 0.5]")
cv2_imshow(rotated)

# 6. 추가 실습 (과제)
아래는 이미지 변환(확대/축소/이동/회전)에 대한 추가 과제입니다. 각 과제마다 직접 코드를 작성/수정하여 결과 이미지를 확인해 보세요.

1. **사용자 입력으로 보간법 선택하기**
   - Python의 `input()` 함수 등을 사용하여 보간법 종류를 문자열로 입력받고, 해당 보간법으로 이미지를 확대/축소해 보세요.
   - 예: "`nearest`, `linear`, `cubic`, `area`, `lanczos4`" 중 택 1.

2. **사용자 입력으로 이동 변환 적용**
   - `tx`, `ty` 값을 사용자가 직접 입력할 수 있게끔 하여 이미지를 움직여 보세요.

3. **사용자 입력으로 회전 변환 적용**
   - 회전 각도(angle), 스케일(scale)을 입력받아 이미지를 회전시키세요.
   - 이미지의 중심이 아닌 **왼쪽 상단**을 기준으로 회전시켜볼 수도 있습니다.

4. **복합 변환**
   - 한 번의 warpAffine으로는 "이동+회전" 또는 "회전+확대" 등 여러 변환을 순차적으로 적용하기 까다로울 수 있습니다.
   - 변환 행렬을 직접 곱해서(행렬 연산) 복합 변환을 만들어 보거나, `warpAffine`을 여러 번 적용해 보세요.

아래 코드 셀에는 간단한 과제 예시풀이가 들어 있습니다. 원하는 과제를 추가하거나 수정하여 실습해보세요.

## 6.1 과제 결과 코드 예시
아래는 간단히 **사용자 입력**을 통해 이미지를 **이동(Translation)** 하도록 한 예시 코드입니다.
원하는 대로 코드를 수정해보세요.

In [ ]:
# (과제 1) 사용자 입력을 통해 이미지 이동
# Colab 환경에서는 input() 사용 시 인터랙티브 입력이 번거로울 수 있으므로
# 여기에선 직접 값을 지정하는 식으로 예시 코드를 보여드립니다.

image_copy = image.copy()
height, width = image_copy.shape[:2]

# x, y 이동 값을 직접 지정 (양수 → 오른쪽/아래, 음수 → 왼쪽/위)
tx = 100  # 원하는 값으로 수정
ty = 50   # 원하는 값으로 수정

M_translate = np.float32([
    [1, 0, tx],
    [0, 1, ty]
])

translated_image = cv2.warpAffine(
    image_copy,
    M_translate,
    (width, height)
)

print(f"[과제 1] 사용자 정의 이동 (x:{tx}, y:{ty}) 적용")
cv2_imshow(translated_image)


## 6.2 추가 과제 아이디어
1. **보간법 사용자 선택**
   - `input()`으로 문자열을 입력받아 `if-elif` 문으로 보간법 변수를 설정해 보세요.
2. **회전 변환에서 중심점 변경**
   - `(0, 0)`을 회전 중심으로 지정하면 어떻게 달라지는지 실습해 보세요.
3. **복합 변환**
   - 회전 행렬과 이동 행렬을 곱해서 한번에 적용하거나, `warpAffine`을 연속 두 번 적용해 보세요.
4. **perspective transform**(난이도 ↑)
   - `cv2.warpPerspective`를 이용해 투시 변환 실습까지 확장해볼 수 있습니다.

실험하면서 스스로 궁금증을 해결해보는 과정이 매우 중요한 학습 방법입니다.

# 7. 참고 자료
- [OpenCV 공식 문서](https://docs.opencv.org/4.x/)  
  - 함수별로 파라미터, 반환값, 예시 코드가 정리되어 있습니다.
- [LearnOpenCV](https://learnopencv.com/)  
  - OpenCV와 딥러닝 관련 다양한 튜토리얼을 볼 수 있는 사이트.

위 자료를 통해 더 많은 예제와 기능을 학습하시길 권장합니다!